# Ghost Attack ~ all frequencies: SLCH018

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import re
import os
import mne
import IPython
import seaborn as sns
import scipy
import joblib
import h5io
import dask.array as da 

import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest

## import custom functions
import sys
sys.path.append('/home/brooke/pacman/preprocessing/scripts')
import preproc_functions as pf
import roi as subs


In [2]:
## Prep paths ##

subject = 'SLCH018'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Neural Data

# load
ghost_attack_epochs = mne.read_epochs(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif")

# get good epochs (for behavioral data only)
good_epochs = [i for i,x in enumerate(ghost_attack_epochs.get_annotations_per_epoch()) if not x]
bad_epochs = [i for i,x in enumerate(ghost_attack_epochs.get_annotations_per_epoch()) if  x]

# load behavioral data
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")

# set info as metadata
ghost_attack_epochs.metadata = ghost_attack_data

# onlt good epochs
ghost_attack_epochs = ghost_attack_epochs[good_epochs]

Reading /home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_ghost_attack_events.fif ...


/tmp/ipykernel_50640/3789543956.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs = mne.read_epochs(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif")


    Found the data of interest:
        t =   -4000.00 ...    6000.00 ms
        0 CTF compensation matrices available
Not setting metadata
41 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 5 columns


In [5]:
ghost_attack_epochs.pick_types(seeg=True).info['ch_names']

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


['FEF2',
 'A61',
 'A62',
 'A63',
 'A64',
 'B61',
 'B62',
 'B63',
 'B64',
 'C34',
 'C35',
 'C36',
 'C37',
 'C38',
 'C39',
 'C40',
 'C41',
 'C42',
 'C43',
 '`A1-`A2',
 '`A2-`A3',
 '`A3-`A4',
 '`A4-`A5',
 '`A5-`A6',
 '`A6-`A7',
 '`A7-`A8',
 '`A8-`A9',
 '`A9-`A10',
 '`A10-`A11',
 '`A11-`A12',
 '`B1-`B2',
 '`B2-`B3',
 '`B3-`B4',
 '`B4-`B5',
 '`B5-`B6',
 '`B6-`B7',
 '`B7-`B8',
 '`B8-`B9',
 '`B9-`B10',
 '`B10-`B11',
 '`B11-`B12',
 '`C4-`C5',
 '`C5-`C6',
 '`C6-`C7',
 '`C7-`C8',
 '`C8-`C9',
 '`C9-`C10',
 '`C10-`C11',
 '`C11-`C12',
 '`C12-`C13',
 '`D1-`D2',
 '`D2-`D3',
 '`D3-`D4',
 '`D4-`D5',
 '`D5-`D6',
 '`E1-`E2',
 '`E2-`E3',
 '`E3-`E4',
 '`E4-`E5',
 '`E5-`E6',
 '`E6-`E7',
 '`E7-`E8',
 '`E8-`E9',
 '`F1-`F2',
 '`F2-`F3',
 '`F3-`F6',
 '`F6-`F7',
 '`F7-`F8',
 '`F8-`F9',
 '`F9-`F10',
 '`G1-`G2',
 '`G2-`G3',
 '`G3-`G4',
 '`G4-`G5',
 '`G5-`G6',
 '`G6-`G7',
 '`G7-`G8',
 '`G8-`G9',
 '`G9-`G10',
 '`G10-`G11',
 '`G11-`G12',
 '`G12-`G13',
 '`G13-`G14',
 '`G14-`G15',
 '`G15-`G16',
 '`H1-`H2',
 '`H2-`H3',


In [4]:
## Dictionary of electrode locations ##

# Pull mapping ROI to elecs
ROIs = subs.ROIs[subject]

## prep lists

# limbic ROI
hc_list = []
hc_indices = []
hc_names = []
ofc_list = []
ofc_indices = []
ofc_names = []
amyg_list = []
amyg_names = [] 
amyg_indices = []
cing_list = []
cing_names = [] 
cing_indices = []
sgACC_list = []
sgACC_names = [] 
sgACC_indices = []
dACC_list = []
dACC_names = [] 
dACC_indices = []
sfg_list = []
sfg_names = [] 
sfg_indices = []
mfg_list = []
mfg_names = [] 
mfg_indices = []

# control ROI
insula_list = []
insula_names = []  
insula_indices = []
dlpfc_list = []
dlpfc_names = []  
dlpfc_indices = []
ec_list = []
ec_names = []  
ec_indices = []

# exclude bad ROI from list
pairs_long_name = [ch.split('-') for ch in ghost_attack_epochs.pick_types(seeg=True).info['ch_names']]
bidx = len(ghost_attack_epochs.info['bads']) +1
pairs_name = pairs_long_name[bidx:len(pairs_long_name)]

# sort ROI into lists
for ix in range(0, len(pairs_name)):
    if pairs_name[ix][0] in ROIs['hc'] or pairs_name[ix][1] in ROIs['hc']:
        hc_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])
        hc_names.append(pairs_name[ix])
        hc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['ofc'] or pairs_name[ix][1] in ROIs['ofc']:
        ofc_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])
        ofc_names.append(pairs_name[ix])
        ofc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['amyg'] or pairs_name[ix][1] in ROIs['amyg']:
        amyg_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        amyg_names.append(pairs_name[ix])
        amyg_indices.append(ix)
    if pairs_name[ix][0] in ROIs['cing'] or pairs_name[ix][1] in ROIs['cing']:
        cing_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        cing_names.append(pairs_name[ix])
        cing_indices.append(ix)
    if pairs_name[ix][0] in ROIs['sgACC'] or pairs_name[ix][1] in ROIs['sgACC']:
        sgACC_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        sgACC_names.append(pairs_name[ix])
        sgACC_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dACC'] or pairs_name[ix][1] in ROIs['dACC']:
        dACC_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        dACC_names.append(pairs_name[ix])
        dACC_indices.append(ix)        
        
    # control roi
    if pairs_name[ix][0] in ROIs['insula'] or pairs_name[ix][1] in ROIs['insula']:
        insula_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        insula_names.append(pairs_name[ix])
        insula_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dlpfc'] or pairs_name[ix][1] in ROIs['dlpfc']:
        dlpfc_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        dlpfc_names.append(pairs_name[ix])
        dlpfc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['sfg'] or pairs_name[ix][1] in ROIs['sfg']:
        sfg_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        sfg_names.append(pairs_name[ix])
        sfg_indices.append(ix)   
    if pairs_name[ix][0] in ROIs['mfg'] or pairs_name[ix][1] in ROIs['mfg']:
        mfg_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        mfg_names.append(pairs_name[ix])
        mfg_indices.append(ix)     
    if pairs_name[ix][0] in ROIs['ec'] or pairs_name[ix][1] in ROIs['ec']:
        ec_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        ec_names.append(pairs_name[ix])
        ec_indices.append(ix)        
        

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


IndexError: list index out of range

In [ ]:
# Print and check frequencies
print(pf.freqs)
print(pf.n_cycles)
print(pf.time_bin)
print(pf.band_width)


# Main Regions of Interest

## Hippocampus

In [ ]:
hc_list

In [ ]:
# only ROI of interest
ghost_attack_roi = ghost_attack_epochs.copy().pick_channels(hc_list)


In [ ]:
# Resample to 1000 
if ghost_attack_roi.info['sfreq'] > 1000:
    ghost_attack_roi= ghost_attack_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = pf.compute_TFR(ghost_attack_roi,pf.freqs,pf.n_cycles)


In [ ]:
# Crop to time of interest

roi_tfr.crop(tmin = -1, tmax = 3)

In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/pacman/preprocessing/{subject}/ieeg/ghost_attack/hc-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = pf.log_and_zscore_TFR(roi_tfr, baseline = (-1,3), logflag=True)

In [ ]:
# Plot TFR
pf.plot_average_tfr(roi_tfr, f"Average Hippocampal TFR at Ghost Attack, n={len(hc_list)}", subject, 'hc_ghostattack')


## Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    pf.plot_channel_tfr(roi_tfr, chix, ch, 'Ghost Attack')
    


# OFC

In [ ]:
ofc_list

In [ ]:
# only roi of interest
ghost_attack_roi = ghost_attack_epochs.copy().pick_channels(ofc_list)


In [ ]:
# Resample to 1000 
if ghost_attack_roi.info['sfreq'] > 1000:
    ghost_attack_roi= ghost_attack_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = pf.compute_TFR(ghost_attack_roi,pf.freqs,pf.n_cycles)

In [ ]:
roi_tfr.crop(tmin = -1, tmax = 3)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/pacman/preprocessing/{subject}/ieeg/ghost_attack/ofc-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = pf.log_and_zscore_TFR(roi_tfr, baseline = (-1,3), logflag=True)

In [ ]:
# Plot TFR
pf.plot_average_tfr(roi_tfr, f"Average OFC TFR at Ghost Attack, n={len(ofc_list)}", subject, 'ofc_ghostattack')


### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    pf.plot_channel_tfr(roi_tfr, chix, ch, 'Ghost Attack')

# Amygdala

In [ ]:
amyg_list

In [ ]:
# only roi 
ghost_attack_roi = ghost_attack_epochs.copy().pick_channels(amyg_list)


In [ ]:
# Resample to 1000 
if ghost_attack_roi.info['sfreq'] > 1000:
    ghost_attack_roi= ghost_attack_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = pf.compute_TFR(ghost_attack_roi,pf.freqs,pf.n_cycles)


In [ ]:
roi_tfr.crop(tmin = -1, tmax = 3)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/pacman/preprocessing/{subject}/ieeg/ghost_attack/amyg-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = pf.log_and_zscore_TFR(roi_tfr, baseline = (-1,3), logflag=True)

In [ ]:
# Plot TFR
pf.plot_average_tfr(roi_tfr, f"Average Amygdala TFR at Ghost Attack, n={len(amyg_list)}", subject, 'amyg_ghostattack')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    pf.plot_channel_tfr(roi_tfr, chix, ch, 'Ghost Attack')